In [2]:
! cat concat.txt

file frames/black_frame.jpg
duration 1.6666666666666667
file frames/white_frame.jpg
duration 1.6666666666666667
file frames/black_frame.jpg
duration 1.6666666666666667
file frames/white_frame.jpg
duration 2.5
file frames/black_frame.jpg
duration 2.5
file frames/black_frame.jpg


In [3]:
! cat concat_count.sh

#!/bin/bash

set -eux

grep duration concat.txt | awk '{ sum += $2 } END { print sum }'


In [6]:
! bash ./concat_count.sh

+ grep duration concat.txt
+ awk '{ sum += $2 } END { print sum }'
10


In [7]:
! cat ./ffconcat.sh

#!/bin/bash

set -eux

ffmpeg -hide_banner -f concat -i concat.txt -fps_mode vfr -pix_fmt yuv420p -c:v libx264 output.mp4 -y
ffprobe -hide_banner -show_streams output.mp4 | grep --color "duration="


In [10]:
! bash ./ffconcat.sh

+ ffmpeg -hide_banner -f concat -i concat.txt -fps_mode vfr -pix_fmt yuv420p -c:v libx264 output.mp4 -y
Input #0, concat, from 'concat.txt':
  Duration: N/A, start: 0.000000, bitrate: N/A
  Stream #0:0: Video: mjpeg (Baseline), yuvj420p(pc, bt470bg/unknown/unknown), 1920x1080 [SAR 1:1 DAR 16:9], 25 fps, 25 tbr, 25 tbn
Stream mapping:
  Stream #0:0 -> #0:0 (mjpeg (native) -> h264 (libx264))
Press [q] to stop, [?] for help
[swscaler @ 0x1280a8000] deprecated pixel format used, make sure you did set range correctly
[swscaler @ 0x138678000] deprecated pixel format used, make sure you did set range correctly
[swscaler @ 0x1180a8000] deprecated pixel format used, make sure you did set range correctly
[swscaler @ 0x120508000] deprecated pixel format used, make sure you did set range correctly
[libx264 @ 0x136608870] using SAR=1/1
[libx264 @ 0x136608870] using cpu capabilities: ARMv8 NEON
[libx264 @ 0x136608870] profile High, level 4.0, 4:2:0, 8-bit
[libx264 @ 0x136608870] 264 - core 164 r3108

**Duration** from ffprobe is not equal to duration we set through `concat.txt` file. We can use `drawtext` filter with `pts` parameter to output frame timestamp.

In [11]:
! cat extract.sh

#!/bin/bash

set -eux

rm -frd ./tmp
mkdir -p ./tmp
ffmpeg -i output.mp4 -vf "drawtext=fontfile=arialbd.ttf:fontsize=24:fontcolor=white:box=1:boxcolor=black@0.5:boxborderw=5:x=(w-tw)/2:y=h-th-10:text='%{pts\:hms}'" -vsync 0 ./tmp/output_frame_%d.png


In [12]:
! bash ./extract.sh

+ rm -frd ./tmp
+ mkdir -p ./tmp
+ ffmpeg -i output.mp4 -vf 'drawtext=fontfile=arialbd.ttf:fontsize=24:fontcolor=white:box=1:boxcolor=black@0.5:boxborderw=5:x=(w-tw)/2:y=h-th-10:text='\''%{pts\:hms}'\''' -vsync 0 ./tmp/output_frame_%d.png
ffmpeg version 7.0.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx2

Frame number 5 must have a timestamp pointing to 7.5 seconds(it's a penultimate frame). However, it has 7.48 as shown below.

![image](./tmp/output_frame_5.png)